In [2]:
! pip install torch-geometric-temporal

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48 kB 2.5 MB/s 
     |████████████████████████████████| 2.1 MB 9.0 MB/s 
     |████████████████████████████████| 467 kB 47.0 MB/s 
  Created wheel for torch-geometric-temporal: filename=torch_geometric_temporal-0.54.0-py3-none-any.whl size=86745 sha256=601cdac7bc6c69981fab19f8c646ab07acf0bee07a4bc9f277a390c6cbab1e8e
  Stored in directory: /root/.cache/pip/wheels/fe/17/9b/5f1b49c6f5ccfbaf216fdfc986959b40bbbc546cf505a16942
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689859 sha256=cac687409204a7ea04c408ab9a700533230cd3fa401f3ca90bdfdfb86e0831ff
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=274491 sha256=f0166547668ae6ddc637656b8a434e1b44c62dd531d

We load our data.

In [3]:
from torch_geometric_temporal.dataset import WindmillOutputSmallDatasetLoader
#from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = WindmillOutputSmallDatasetLoader()
#loader = ChickenpoxDatasetLoader()

dataset = loader.get_dataset()

We look at our data.

**How many data points?**

**What does a single datapoint look like?**

**What are the edge indices?**

**What are the edge attributes?**


**What kind of task is this - regression or classification?**

In [4]:
print("Number of data points: ", dataset.snapshot_count)
snapshot = dataset[0]
print("Features: ", snapshot.num_features, "Edge index: ", len(snapshot.edge_index), "Edge attributes: ", len(snapshot.edge_attr))

Number of data points:  17464
Features:  8 Edge index:  2 Edge attributes:  121


We split it into train and test. **Note need to be careful with temporal data.**

In [5]:
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

We define a model.

**What type of model?**

In [6]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = DCRNN(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

We train the model.

**What optimizer?**

**How many epochs?**

In [9]:
from tqdm import tqdm

model = RecurrentGCN(node_features = 8)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(25)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()
    print("MSE: {:.4f}".format(cost))

  4%|▍         | 1/25 [00:31<12:29, 31.24s/it]

MSE: 1.0152


  8%|▊         | 2/25 [01:03<12:06, 31.58s/it]

MSE: 1.0072


 12%|█▏        | 3/25 [01:34<11:33, 31.53s/it]

...

MSE: 1.0003


 96%|█████████▌| 24/25 [12:36<00:31, 31.04s/it]

MSE: 1.0001


100%|██████████| 25/25 [13:07<00:00, 31.50s/it]

MSE: 0.9999


We evaluate.

**What is MSE?**

In [10]:
model.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

MSE: 0.9856
